In [3]:
from osgeo import gdal
import numpy as np

In [4]:
MODIS_min = 0
MODIS_max = 10000

S1_min = -50
S1_max = 1

S2_min = 0
S2_max = 10000

In [5]:
def tif2npy(tif_path, output_path, image_class):
    ds = gdal.Open(tif_path)
    if ds is None:
        print("No such file !")

    image = ds.ReadAsArray(0, 0, ds.RasterXSize, ds.RasterYSize)

    if image_class in ["MODIS_l", "MODIS_h"]:
        if image_class == "MODIS_l":
            image = image[[0, 1, 2, 3, 5, 6], :, :]
        # 有效范围截断
        np.clip(image, 0, 10000, out=image)
        image = (image - MODIS_min) / (MODIS_max - MODIS_min)
    elif image_class == "S1":
        image = image[[0, 1], :, :]
        # 有效范围截断
        np.clip(image, -50, 1, out=image)
        image = (image - S1_min) / (S1_max - S1_min)
    elif image_class in ["S2", "ref", "before", "after"]:
        image = image[[1, 2, 3, 4, 7, 8, 10, 11], :, :]
        # 有效范围截断
        np.clip(image, 0, 10000, out=image)
        image = (image - S2_min) / (S2_max - S2_min)

    # S2: (8, 250, 250), S1: (2, 250, 250), MODIS: (6, 5, 5)
    np.save(output_path, image.astype(np.float32))

    ds = None

    print("Successfully convert", tif_path, "to", output_path)
    
    return True

In [6]:
import os

def get_files_by_type(folder_path, file_type):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(file_type):
                file_paths.append(os.path.join(root, file))
    return file_paths

def categorize_files_by_keyword(file_paths, keywords):
    categorized_files = {keyword: [] for keyword in keywords}
    
    for file_path in file_paths:
        for keyword in keywords:
            if keyword in file_path:
                categorized_files[keyword].append(file_path)
    
    return categorized_files

folder_path = r"D:\ENVI\data\NingBo\NB_roi\cropped_data"

file_type = ".tif"

# keywords = ["MODIS_l", "MODIS_h", "S1", "S2", "ref", "before", "after"]
keywords = ["MODIS_h", "MODIS_l"]

file_paths = get_files_by_type(folder_path, file_type)

categorized_files = categorize_files_by_keyword(file_paths, keywords)

# for keyword, files in categorized_files.items():
#     print(f"Files with '{keyword}' in the path:")
#     for file in files:
#         print(file)
#     print("\n")


In [7]:
for keyword, files in categorized_files.items():
    for file_path in files:
        output_path = file_path.replace(r"D:\ENVI\data\NingBo\NB_roi\cropped_data", r"D:\Code\modis-s1-s2\dataset\SatelliteImages").replace("tif", "npy")
        tif2npy(file_path, output_path, image_class=keyword)

Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS_h\MODIS_2194.tif to D:\Code\modis-s1-s2\dataset\SatelliteImages\test\MODIS_h\MODIS_2194.npy
Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS_h\MODIS_2195.tif to D:\Code\modis-s1-s2\dataset\SatelliteImages\test\MODIS_h\MODIS_2195.npy
Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS_h\MODIS_2196.tif to D:\Code\modis-s1-s2\dataset\SatelliteImages\test\MODIS_h\MODIS_2196.npy
Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS_h\MODIS_2197.tif to D:\Code\modis-s1-s2\dataset\SatelliteImages\test\MODIS_h\MODIS_2197.npy
Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS_h\MODIS_2198.tif to D:\Code\modis-s1-s2\dataset\SatelliteImages\test\MODIS_h\MODIS_2198.npy
Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS_h\MODIS_2199.tif to D:\Code\modis-s1-s2\dataset\SatelliteImages\test\MODIS_h\MODIS_2199.npy
Successfully con

In [9]:
import glob
import numpy as np


def calc_statistics(image_paths):
    images = []
    for image_path in image_paths:
        images.append(np.load(image_path))

    min_val = np.min(images, axis=(0, 2, 3))
    max_val = np.max(images, axis=(0, 2, 3))
    mean = np.mean(images, axis=(0, 2, 3))
    std = np.std(images, axis=(0, 2, 3))

    return min_val, max_val, mean, std


MODIS_500_dir = r"D:\Code\modis-s1-s2\dataset\SatelliteImages\train\MODIS_l\MODIS_*.npy"
MODIS_250_dir = r"D:\Code\modis-s1-s2\dataset\SatelliteImages\train\MODIS_h\MODIS_*.npy"
S1_dir = r"D:\Code\modis-s1-s2\dataset\SatelliteImages\train\S1\S1_*.npy"
S2_dir = r"D:\Code\modis-s1-s2\dataset\SatelliteImages\train\S2\S2_*.npy"

MODIS_500_image_paths = glob.glob(MODIS_500_dir)
MODIS_250_image_paths = glob.glob(MODIS_250_dir)
S1_image_paths = glob.glob(S1_dir)
S2_image_paths = glob.glob(S2_dir)


# min_val, max_val, mean, std = calc_statistics(MODIS_500_image_paths)
# print("MODIS_500: ")
# print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

min_val, max_val, mean, std = calc_statistics(MODIS_250_image_paths)
print("MODIS_250: ")
print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

# min_val, max_val, mean, std = calc_statistics(S1_image_paths)
# print("S1: ")
# print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

# min_val, max_val, mean, std = calc_statistics(S2_image_paths[:1500])
# print("S2: ")
# print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

MODIS_250: 
min: [0.     0.0224]
max: [0.9338 1.    ]
mean: [0.08833838 0.24816364]
std: [0.05251674 0.07744461]


In [2]:
import numpy as np

image = np.load(r"D:\Code\MODIS-S1-S2\dataset\SatelliteImages\train\MODIS_l\MODIS_250.npy")
print(image.shape)

min_val = np.min(image, axis=(1, 2))
max_val = np.max(image, axis=(1, 2))
mean = np.mean(image, axis=(1, 2))
std = np.std(image, axis=(1, 2))


print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

(7, 5, 5)
min: [0.0859 0.1985 0.0486 0.0777 0.2158 0.1872 0.1182]
max: [0.12   0.3127 0.0652 0.1038 0.2676 0.2349 0.1649]
mean: [0.1018     0.23174798 0.056256   0.090776   0.23836401 0.20572
 0.14074   ]
std: [0.00820366 0.03034872 0.00473541 0.00689941 0.01195966 0.01223699
 0.0130241 ]
